# A transformer-based model based on "What does self-attention learn from Masked Language Modelling?" paper

### Core parts of the transformer
- Separated position and spin
- Single attention layer


### Outline
1. Vanilla attention implementation
2. Factored attention implementation


In [320]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
import numpy as np
import tqdm
import random
import torch.nn.functional as F

In [321]:
X = torch.randn(100,1000,300)
idx = torch.randint(0,1000,(100,))
# create masked_X, Y=model(masked_X)
# X[:,idx,:]-Y[:,idx,:]



In [322]:
data_train = torch.tensor([np.random.choice([-1, 1], size=20) for _ in range(1000)])
data_test = torch.tensor([np.random.choice([-1, 1], size=20) for _ in range(100)])

def one_hot_encoding(seq, vocab):
    if isinstance(seq, int):
        one_hot = torch.zeros(1, len(vocab), dtype=int)
        one_hot[:, vocab[seq]] = 1
    else:
        one_hot = torch.zeros((len(seq), len(vocab)), dtype=int)
        for i, spin in enumerate(seq):
            one_hot[i, vocab[spin]] = 1
    return one_hot

def mask_random_spin(sequence, mask_token=2):
    """
    Mask one random spin in a sequence of protein spins.
    
    Parameters:
    - sequence: a list or sequence of spins (integers)
    - mask_token: the token used to mask a spin (default is 2)
    
    Returns:
    - masked_sequence: a sequence similar to the input but with one spin masked
    - masked_position: the position of the spin that was masked
    """
    # define vocabulary
    vocab = {-1:0,1:1,2:2}
    # Ensure the sequence can be converted to a list for masking
    sequence_list = sequence.numpy().tolist() if isinstance(sequence, torch.Tensor) else list(sequence)
    
    # Choose a random position to mask, excluding the first spin
    mask_position = random.randint(1, len(sequence_list) - 1)
    
    # Mask the chosen position
    masked_sequence = sequence_list.copy()
    masked_sequence[mask_position] = mask_token

    # Create an array of zeros with shape (len(sequence), len(vocab))
    #one_hot = np.zeros((len(sequence_list), len(vocab)), dtype=int)
    #for i, spin in enumerate(masked_sequence):
    #    one_hot[i, vocab[spin]] = 1
    one_hot = one_hot_encoding(masked_sequence, vocab)
    # Display the one-hot encoding of the masked spin
    #mask_encoding = np.zeros((1, len(vocab)), dtype=int)
    #mask_encoding[0, vocab[2]] = 1
        
    return torch.tensor(one_hot), torch.tensor(mask_position)
    

In [327]:
class VanillaAttentionTransformer(nn.Module):
    def __init__(self, embed_dim, a, max_seq_length, num_spins=3, dropout_rate=0.0):
        super(VanillaAttentionTransformer, self).__init__()
        self.word_embeddings = nn.Linear(num_spins, embed_dim)
        self.position_embeddings = nn.Embedding(max_seq_length, embed_dim)
        self.a = a  # parameter controlling how important positions are
        self.value_weight = nn.Linear(embed_dim, embed_dim)
        self.query_weight = nn.Linear(embed_dim, embed_dim)
        self.key_weight = nn.Linear(embed_dim, embed_dim)
        self.fc = nn.Linear(embed_dim, num_spins)  # output layer
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, s, masked_token):
        #if not isinstance(s, torch.Tensor):
        s = torch.tensor(s, dtype=torch.float)
        #if not isinstance(masked_token, torch.Tensor):
        masked_token = torch.tensor(masked_token, dtype=torch.float)

        position_ids = torch.arange(s.size(0), dtype=torch.long)
        x = self.word_embeddings(s) + self.a*self.position_embeddings(position_ids)
        
        query = self.query_weight(x)
        key = self.key_weight(x)
        values = self.value_weight(x)
        
        # Simple attention score calculation (Dot product): this is equivalent to the interaction matrix
        scores = torch.matmul(query, key.transpose(-2, -1))  # Transpose last two dimensions for matrix multiplication
        scores = torch.softmax(scores, dim=-1)  # Apply softmax to scores to get probabilities

        # Apply attention scores to values
        attn_output = torch.matmul(scores, values)

        # Sum over the sequence length dimensions
        #print("attention before summing:", attn_output.shape)
        attn_output = attn_output.sum(dim=0)
        #print("attention after summing:", attn_output.shape)
        output = self.fc(self.dropout(attn_output))

        return output


In [328]:
def evaluate(model, data_test, vocab, criterion, device=0):
    model.eval()
    epoch_loss = 0
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    for i, data in tqdm.tqdm(enumerate(data_test), total=len(data_test)):
        # Get the inputs
        input_seq = data
        input_one_hot = one_hot_encoding(input_seq.tolist(), vocab)
        # mask a token
        masked_sequence, position = mask_random_spin(input_seq, mask_token=2)
        # Forward pass
        outputs = model.forward(masked_sequence, masked_sequence[position])

        #output_token = F.log_softmax(outputs, dim=-1)
        target_token = torch.argwhere(input_one_hot[position]==1).squeeze(0) #target_token = input_seq[position]

        # Compute loss
        loss = criterion(outputs.unsqueeze(0), target_token)
        epoch_loss += loss.item()
    #model.train()
    return epoch_loss / len(data_test)

def train(model, data_train, data_test, vocab, optimizer, criterion, num_epochs=8, device=0):
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    # Training loop
    model.train()
    best_eval_loss = 1e-3 # used to do early stopping

    for epoch in tqdm.tqdm(range(num_epochs), leave=False, position=0):
        running_loss = 0
        epoch_loss = 0
        
        for i, data in tqdm.tqdm(enumerate(data_train), total=len(data_train)):
            # Get the inputs
            input_seq = data
            input_one_hot = one_hot_encoding(input_seq.tolist(), vocab)

            # mask a token
            masked_sequence, position = mask_random_spin(input_seq, mask_token=2)

            # Forward pass
            prediction = model.forward(masked_sequence, masked_sequence[position]) #masked_sequence[masked_position]
            
            #predicted_token = F.log_softmax(prediction, dim=-1)
            target_token = torch.argwhere(input_one_hot[position]==1).squeeze(0) #input_seq[masked_position]
            # Compute loss
            #print("predicted_token:", predicted_token)
            #print("predicted_token:", prediction)
            #print("target token:", target_token)
            loss = criterion(prediction.unsqueeze(0), target_token)
            epoch_loss += loss.item()
            
            # Zero gradients, perform a backward pass, and update the weights.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if i % 10 == 9. :    # print every 10 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0

        print(f'Epoch {epoch + 1} | Train Loss: {(epoch_loss / len(data)):.4f}')
        eval_loss = evaluate(model, data_test, vocab, criterion, device=device)
        print(f'Epoch {epoch + 1} | Eval Loss: {(eval_loss):.4f}')
        
        # Perform early stopping based on eval loss
        if eval_loss < best_eval_loss:
            return epoch_loss / len(data_train)
    return epoch_loss / len(data_train)



In [329]:
# Define the parameters 
vocab_size = 3
vocab = {-1:0,1:1,2:2} 
L = 20
embedding_dim = 20
hidden_dim = 20
num_layers = 1 # have to adapt the model for 2 and 3 layers
dropout_rate = 0.0
lr = 1e-3
num_sequences = 1000
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [330]:
# Example usage:
model = VanillaAttentionTransformer(embed_dim=embedding_dim, a=0, max_seq_length=L, num_spins=3, dropout_rate=dropout_rate)

optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

train(model, data_train, data_test, vocab, optimizer, criterion, device=device)
#torch.save(model.state_dict(), 'models/lstm_scratch.pt')
#evaluate(model, test_dataloader, criterion, device=device)

  0%|          | 0/8 [00:00<?, ?it/s]/var/folders/mg/44g5ch495hdc8x4_5mq3dsdm0000gn/T/ipykernel_82546/2066114617.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(one_hot), torch.tensor(mask_position)
/var/folders/mg/44g5ch495hdc8x4_5mq3dsdm0000gn/T/ipykernel_82546/10981384.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float)
/var/folders/mg/44g5ch495hdc8x4_5mq3dsdm0000gn/T/ipykernel_82546/10981384.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  masked_token = torch.tensor

[1,    10] loss: 0.136
[1,    20] loss: 0.089
[1,    30] loss: 0.080
[1,    40] loss: 0.056
[1,    50] loss: 0.092
[1,    60] loss: 0.068
[1,    70] loss: 0.073
[1,    80] loss: 0.071
[1,    90] loss: 0.077
[1,   100] loss: 0.067
[1,   110] loss: 0.080
[1,   120] loss: 0.065
[1,   130] loss: 0.078
[1,   140] loss: 0.066
[1,   150] loss: 0.075
[1,   160] loss: 0.061
[1,   170] loss: 0.082
[1,   180] loss: 0.079
[1,   190] loss: 0.071
[1,   200] loss: 0.073
[1,   210] loss: 0.066
[1,   220] loss: 0.085
[1,   230] loss: 0.090
[1,   240] loss: 0.081
[1,   250] loss: 0.086
[1,   260] loss: 0.065
[1,   270] loss: 0.072
[1,   280] loss: 0.070
[1,   290] loss: 0.070
[1,   300] loss: 0.066
[1,   310] loss: 0.070
[1,   320] loss: 0.074
[1,   330] loss: 0.072
[1,   340] loss: 0.050
[1,   350] loss: 0.101
[1,   360] loss: 0.062
[1,   370] loss: 0.073
[1,   380] loss: 0.067
[1,   390] loss: 0.085


[1,   400] loss: 0.079
[1,   410] loss: 0.072
[1,   420] loss: 0.072
[1,   430] loss: 0.069
[1,   440] loss: 0.069
[1,   450] loss: 0.086
[1,   460] loss: 0.075
[1,   470] loss: 0.069
[1,   480] loss: 0.094
[1,   490] loss: 0.071
[1,   500] loss: 0.089
[1,   510] loss: 0.083
[1,   520] loss: 0.047
[1,   530] loss: 0.081
[1,   540] loss: 0.078
[1,   550] loss: 0.063
[1,   560] loss: 0.073
[1,   570] loss: 0.112
[1,   580] loss: 0.082
[1,   590] loss: 0.079
[1,   600] loss: 0.073
[1,   610] loss: 0.075
[1,   620] loss: 0.101
[1,   630] loss: 0.075
[1,   640] loss: 0.079
[1,   650] loss: 0.063
[1,   660] loss: 0.093
[1,   670] loss: 0.069
[1,   680] loss: 0.059
[1,   690] loss: 0.105
[1,   700] loss: 0.070
[1,   710] loss: 0.056
[1,   720] loss: 0.087
[1,   730] loss: 0.075
[1,   740] loss: 0.070
[1,   750] loss: 0.072
[1,   760] loss: 0.072
[1,   770] loss: 0.069
[1,   780] loss: 0.067
[1,   790] loss: 0.083
[1,   800] loss: 0.067


100%|██████████| 1000/1000 [00:00<00:00, 1911.70it/s]


[1,   810] loss: 0.070
[1,   820] loss: 0.071
[1,   830] loss: 0.072
[1,   840] loss: 0.067
[1,   850] loss: 0.073
[1,   860] loss: 0.073
[1,   870] loss: 0.065
[1,   880] loss: 0.072
[1,   890] loss: 0.069
[1,   900] loss: 0.075
[1,   910] loss: 0.065
[1,   920] loss: 0.080
[1,   930] loss: 0.079
[1,   940] loss: 0.076
[1,   950] loss: 0.069
[1,   960] loss: 0.084
[1,   970] loss: 0.069
[1,   980] loss: 0.072
[1,   990] loss: 0.077
[1,  1000] loss: 0.065
Epoch 1 | Train Loss: 37.5421


...

100%|██████████| 1000/1000 [00:00<00:00, 1837.94it/s]


[8,   990] loss: 0.068
[8,  1000] loss: 0.070
Epoch 8 | Train Loss: 34.7998


100%|██████████| 8/8 [00:04<00:00,  1.69it/s]

Epoch 8 | Eval Loss: 0.6881


0.6959957875907421

# Questions
- Do we pass through the sequence multiple times, each time masking different tokens or do we pass through the sequence once masking one random token?
- Maybe we evaluate on how well the model predicts the other token in the sequence?

## Previous implementations

In [198]:
class VanillaAttentionTransformer(nn.Module):
    def __init__(self, embed_dim, a, max_seq_length, num_spins=3, dropout_rate=0.0):
        super(VanillaAttentionTransformer, self).__init__()
        self.word_embeddings = nn.Linear(num_spins, embed_dim)
        self.position_embeddings = nn.Embedding(max_seq_length, embed_dim)
        self.a = a # parameter controlling how important are positions
        self.value_weight = nn.Linear(embed_dim, embed_dim)
        self.query_weight = nn.Linear(embed_dim, embed_dim)
        self.key_weight = nn.Linear(embed_dim, embed_dim)
        self.fc = nn.Linear(embed_dim, num_spins) # output layer
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, s, masked_token):
        position_ids = torch.arange(len(s), dtype=torch.long)
        position_ids = position_ids
        s = torch.tensor(s, dtype=torch.float)
        masked_token = torch.tensor(masked_token, dtype=torch.float)
        #print("the sequence with the masked token:", s)
        print("shape of position ids before embedding:", position_ids.shape)
        print("shape the sequence with the masked token:", s.shape)
        print("value of the word embedding:", self.word_embeddings(s).shape)
        print("positional embedding:", self.position_embeddings(position_ids))
        x = self.word_embeddings(s) + self.position_embeddings(position_ids)
        query = self.query_weight(x)
        key = self.key_weight(x)

        values = self.value_weight(self.word_embeddings(s) + self.a*self.position_embeddings(position_ids)) # (batch_size,embed_dim)
        exp_scaling = torch.exp(self.word_embeddings(masked_token) + self.position_embeddings(position_ids).T@query.T@key@(self.word_embeddings(s) + self.a*self.position_embeddings(position_ids)))
        attn_output = torch.sum(exp_scaling/torch.sum(exp_scaling.sum(0))*values) # not sure it multiplies the way I want it to multiply - check
        output = self.dropout(self.fc(attn_output))
        return output
    
class FactoredAttentionTransformer(nn.Module):
    def __init__(self, embed_dim, a, max_seq_length, num_spins=3, dropout_rate=0.0):
        super(FactoredAttentionTransformer, self).__init__()
        self.word_embeddings = nn.Embedding(num_spins, embed_dim)
        self.word_embedding.weight.requires_grad = False
        self.position_embeddings = nn.Embedding(max_seq_length, embed_dim)
        self.a = a # parameter controlling how important are positions
        self.value_weight = nn.Linear(embed_dim, embed_dim)
        self.query_weight = nn.Linear(embed_dim, embed_dim)
        self.key_weight = nn.Linear(embed_dim, embed_dim)
        self.fc = nn.Linear(embed_dim, num_spins) # output layer
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, s, masked_token):
        # masked token should be equal to 0
        # masked_token = torch.tensor([0])
        position_ids = torch.arange(len(s), dtype=torch.long)
        position_ids = position_ids
        x = self.word_embeddings(s) + self.position_embeddings(position_ids)
        query = self.query_weight(x)
        key = self.key_weight(x)

        values = self.value_weight(self.word_embeddings(s)) # (embed_dim)
        exp_scaling = torch.exp(self.word_embeddings(masked_token) + self.position_embeddings(position_ids).T@query.T@key@(self.word_embeddings(s) + self.a*self.position_embeddings(position_ids)))
        attn_output = torch.sum(exp_scaling/torch.sum(exp_scaling.sum(0))*values) # not sure it multiplies the way I want it to multiply - check
        output = self.fc(self.dropout(attn_output))
        return output